In [ ]:
# https://hackernoon.com/caches-in-python
# https://www.scaler.com/topics/python-cache/

##### Dictionary-Based Caching

In [172]:
# Initialize an empty cache dictionary
cache = {}

# Function to calculate the factorial of a number
def factorial(n):
    if n in cache:
        print(f"cache:: {cache}")
        return cache[n]  # Return cached result if available

    # Calculate factorial if not cached
    result = 1
    for i in range(1, n + 1):
        result *= i

    # Cache the result
    cache[n] = result
    return result

# Example usage
print(factorial(5))  # Calculates and caches factorial of 5


120


In [173]:
# Using Decorators

def memoize(func):
    cache = {}

    def memoized_func(*args):
        if args not in cache:
            cache[args] = func(*args)
        return cache[args]

    return memoized_func


@memoize
def factorial(n):
    return 1 if n == 0 else n * factorial(n - 1)


print(factorial(5))

120


##### JSON-Based Caching

In [174]:
# Using Decorators

import json

def memoize(func):
    cache_file = 'cache.json'

    try:
        with open(cache_file, 'r') as f:
            cache = json.load(f)
    except FileNotFoundError:
        cache = {}

    def memoized_func(*args):
        key = str(args[0])
        if key not in cache:
            result = func(*args)
            cache[key] = result
            with open(cache_file, 'w') as f:
                json.dump(cache, f)
        return cache[key]

    return memoized_func



@memoize
def factorial(n):
    return 1 if n == 0 else n * factorial(n - 1)


print(factorial(5))

{'result': 120, 'time': '2024-04-02 23:04:17.824119'}


In [175]:
# Using Decorators

from datetime import datetime, timedelta
import json

def memoize(func=None, maxsize=None, typed=None, ttl=None):
    cache_file = 'cache.json'

    try:
        with open(cache_file, 'r') as f:
            cache = json.load(f)
    except FileNotFoundError:
        cache = {}

    def memoized_func(*args, **kwargs):
        # key = args if typed else (args, frozenset(kwargs.items()))
        key = str(args[0])
        # key = str(key)  # Convert to string for key
        if key not in cache:
            result = func(*args, **kwargs)
            cache[key] = {'result': result, 'time': str(datetime.now())}
            with open(cache_file, 'w') as f:
                json.dump(cache, f)
        elif ttl and datetime.now() - cache[key]['time'] > timedelta(seconds=ttl):
            del cache[key]
            result = func(*args, **kwargs)
            cache[key] = {'result': result, 'time': str(datetime.now())}
            with open(cache_file, 'w') as f:
                json.dump(cache, f)
        return cache[key]['result']

    return memoized_func



@memoize
def factorial(n):
    return 1 if n == 0 else n * factorial(n - 1)


print(factorial(5))

120


##### Functools LRU Cache
- It is part of the Python standard library, so no external installation is required.
- -It supports caching with a maximum size limit, eviction of least recently used entries, and memoization of function results.
- - It is easy to use and integrates well with existing functions via decorators

In [176]:
from functools import lru_cache

# With typed=True, you can now set the expiration time using the ttl (time to live) argument

@lru_cache(maxsize=128, typed=False)
def factorial(n):
    return 1 if n == 0 else n * factorial(n - 1)

# Test the cached factorial function
print(factorial(5))  # Calls the function and caches the result
print(factorial.cache_info())
print(factorial.cache_parameters())

120
CacheInfo(hits=0, misses=6, maxsize=128, currsize=6)
{'maxsize': 128, 'typed': False}


- To implement TTL (Time To Live) for caching with functools.lru_cache, you'll need to customize the caching mechanism slightly. 
- Unfortunately, the functools.lru_cache decorator does not directly support TTL. 

In [177]:
import functools
import time

def lru_cache_with_ttl(maxsize=128, typed=False, ttl=None):
    def decorator(func):
        cache = functools.lru_cache(maxsize=maxsize, typed=typed)

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            key = args + tuple(kwargs.items())
            if key in wrapper.cache:
                value, timestamp = wrapper.cache[key]
                if ttl and time.time() > timestamp:
                    del wrapper.cache[key]
                else:
                    return value

            result = func(*args, **kwargs)
            wrapper.cache[key] = (result, time.time() + ttl) if ttl else result
            return result

        def cache_info():
            hits = misses = maxsize = currsize = len(wrapper.cache)
            return functools._CacheInfo(hits, misses, maxsize, currsize)

        def cache_clear():
            wrapper.cache.clear()

        wrapper.cache_info = cache_info
        wrapper.cache_clear = cache_clear
        wrapper.cache = {}

        return wrapper

    return decorator

@lru_cache_with_ttl(maxsize=128, typed=False, ttl=5)  # Set TTL to 5 seconds
def factorial(n):
    return 1 if n == 0 else n * factorial(n - 1)

# Test the cached factorial function
print(factorial(5))  # Calls the function and caches the result
print(factorial.cache_info())


120
CacheInfo(hits=6, misses=6, maxsize=6, currsize=6)


##### Functools cache
- Returns the same as lru_cache(maxsize=None), creating a thin wrapper around a dictionary lookup for the function arguments. Because it never needs to evict old values, this is smaller and faster than lru_cache() with a size limit.
-  It's designed to provide a simple and efficient caching mechanism, but without built-in support for expiration policies.
-  Work only in above python version of 3.9


In [178]:
from functools import cache

@cache
def factorial_1(n):
    return n * factorial(n-1) if n else 1


print(factorial(10))      # no previously cached result, makes 11 recursive calls
print(factorial(5))       # just looks up cached value result
print(factorial(12))      # makes two new recursive calls, the other 10 

print(factorial_1.cache_info())

3628800
120
479001600
CacheInfo(hits=0, misses=0, maxsize=None, currsize=0)


-  can implement a custom caching decorator that adds TTL functionality similar to what we did with functools.lru_cache

In [179]:
import functools
import time

def cache_with_ttl(ttl=None):
    cache = {}

    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            key = args + tuple(kwargs.items())
            if key in cache:
                value, timestamp = cache[key]
                if ttl and time.time() > timestamp:
                    del cache[key]
                else:
                    return value

            result = func(*args, **kwargs)
            cache[key] = (result, time.time() + ttl) if ttl else result
            return result

        def cache_info():
            hits = misses = len(cache)
            return functools._CacheInfo(hits, misses, None, None)

        def cache_clear():
            cache.clear()

        wrapper.cache_info = cache_info
        wrapper.cache_clear = cache_clear

        return wrapper

    return decorator

@cache_with_ttl(ttl=60)  # Set TTL to 60 seconds
def factorial(n):
    return 1 if n == 0 else n * factorial(n - 1)

# Test the cached factorial function
print(factorial(5))  # Calls the function and caches the result
print(factorial.cache_info())


120
CacheInfo(hits=6, misses=6, maxsize=None, currsize=None)


##### Custom Cache Classes
- For more control over caching behavior, you can create custom cache classes that implement specific caching strategies, such as FIFO (First-In-First-Out), LRU, or LFU (Least Frequently Used). 

In [180]:
from collections import OrderedDict

class LRUCache:
    def __init__(self, max_size):
        self.max_size = max_size
        self.cache = OrderedDict()

    def get(self, key):
        if key in self.cache:
            value = self.cache.pop(key)
            self.cache[key] = value  # Move the item to the end to mark it as most recently used
            return value
        else:
            return None

    def put(self, key, value):
        if key in self.cache:
            self.cache.pop(key)
        elif len(self.cache) >= self.max_size:
            self.cache.popitem(last=False)  # Remove the least recently used item
        self.cache[key] = value

def factorial(n, cache):
    if n == 0 or n == 1:
        return 1
    if n in cache.cache:  # Access the cache dictionary inside the LRUCache instance
        return cache.get(n)
    result = n * factorial(n - 1, cache)
    cache.put(n, result)  # Store the result in the cache
    return result

# Example usage:
cache = LRUCache(5)  # You can adjust the cache size as per your requirements
n = 5
print(factorial(n, cache))  # Output: 120


120


- To add caching expiration, you would need to introduce additional logic to track the time each item was added to the cache and implement a mechanism to remove items that have exceeded their expiration time.

In [181]:
from collections import OrderedDict
import time

class LRUCache:
    def __init__(self, max_size, expiration_time):
        self.max_size = max_size
        self.expiration_time = expiration_time
        self.cache = OrderedDict()
        self.timestamps = {}

    def get(self, key):
        if key in self.cache:
            if time.time() - self.timestamps[key] <= self.expiration_time:
                value = self.cache.pop(key)
                self.cache[key] = value  # Move the item to the end to mark it as most recently used
                self.timestamps[key] = time.time()
                return value
            else:
                # Expired, remove from cache
                del self.cache[key]
                del self.timestamps[key]
        return None

    def put(self, key, value):
        current_time = time.time()
        if key in self.cache:
            self.cache.pop(key)
            del self.timestamps[key]
        elif len(self.cache) >= self.max_size:
            # Remove the least recently used item
            oldest_key = next(iter(self.cache))
            del self.cache[oldest_key]
            del self.timestamps[oldest_key]
        self.cache[key] = value
        self.timestamps[key] = current_time



def factorial(n, cache):
    if n == 0 or n == 1:
        return 1
    if n in cache.cache:  # Access the cache dictionary inside the LRUCache instance
        return cache.get(n)
    result = n * factorial(n - 1, cache)
    cache.put(n, result)  # Store the result in the cache
    return result

# Example usage:
cache = LRUCache(5, expiration_time=60)  # Cache size is 5 and expiration time is 60 seconds
n = 5
print(factorial(n, cache))  # Output: 120


120


##### Third-Party Libraries
- cachetools, redis, memcached
- These libraries offer more advanced features and support distributed caching, persistent storage, and scalability.

###### cachetools
- cachetools is a third-party library providing various caching utilities, including LRUCache, TTLCache, LFUCache, and others. 
- It is not part of the Python standard library and needs to be installed separately (pip install cachetools).
- It offers more advanced caching algorithms and configurations compared to functools.lru_cache
  -  For example, it provides different cache implementations such as TTL (time-to-live) cache and LFU (Least Frequently Used) cache.

In [182]:
from cachetools import cached, TTLCache

# Define a TTLCache with a maximum size of 128 and TTL of 60 seconds
cache = TTLCache(maxsize=128, ttl=60)

# Decorate a function to use the cache
@cached(cache)
def factorial(n):
    return 1 if n == 0 else n * factorial(n - 1)

# Test the cached factorial function
print(factorial(5))  # Calls the function and caches the result
print(factorial(4))  # Retrieves the cached result
print(factorial(6))  # Calls the function and caches the result


ModuleNotFoundError: No module named 'cachetools'

###### Redis
- Redis is an in-memory data structure store used as a database, cache, and message broker.

In [183]:
import redis

# Connect to Redis
redis_client = redis.Redis(host='localhost', port=6379, db=0)

# Define a function to use Redis as a cache
def factorial(n):
    cached_result = redis_client.get(f'factorial:{n}')
    if cached_result:
        return int(cached_result)
    result = 1 if n == 0 else n * factorial(n - 1)
    redis_client.set(f'factorial:{n}', result)
    return result

# Test the cached factorial function
print(factorial(5))  # Calls the function and caches the result
print(factorial(4))  # Retrieves the cached result
print(factorial(6))  # Calls the function and caches the result

ModuleNotFoundError: No module named 'redis'

###### Memcached
- Memcached is a high-performance, distributed memory object caching system.

In [184]:
from pymemcache.client.base import Client

# Connect to Memcached
memcached_client = Client(('localhost', 11211))

# Define a function to use Memcached as a cache
def factorial(n):
    cached_result = memcached_client.get(f'factorial:{n}')
    if cached_result:
        return int(cached_result.decode())
    result = 1 if n == 0 else n * factorial(n - 1)
    memcached_client.set(f'factorial:{n}', str(result))
    return result

# Test the cached factorial function
print(factorial(5))  # Calls the function and caches the result
print(factorial(4))  # Retrieves the cached result
print(factorial(6))  # Calls the function and caches the result


ModuleNotFoundError: No module named 'pymemcache'

In [ ]:
# Without LRU Cache
import time
start_time = time.time_ns()
factorial(50)
print(time.time_ns() - start_time)


# With LRU Cache
start_time = time.time_ns()
factorial_lru_cache(50)
print(time.time_ns() - start_time)


In [ ]:
start_time = time.time()
fibonacci(40)
end_time = time.time()
execution_time_without_cache = end_time - start_time
print("Time taken without cache: {:.8f} seconds".format(execution_time_without_cache))

start_time = time.time()
fibonacci(40)
end_time = time.time()
execution_time_with_cache = end_time - start_time
print("Time taken with cache: {:.8f} seconds".format(execution_time_with_cache))